In [88]:
print("all ok")

all ok


In [89]:
from langchain_groq import ChatGroq

In [90]:
from dotenv import load_dotenv

In [91]:
load_dotenv()

True

In [92]:
llm=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [93]:
#print(llm.invoke("What is the capital of France?").context)

In [94]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [95]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [96]:
embedding_model.embed_query("What is the capital of France?")

GoogleGenerativeAIError: Error embedding content: 403 Your API key was reported as leaked. Please use another API key.

### 1. Data Ingestion

In [ ]:
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
import os

In [ ]:
file_path= os.path.join(os.getcwd(), "data", "sample.pdf")

In [ ]:
loader = PyPDFLoader(file_path)

In [ ]:
documents = loader.load()

In [ ]:
documents

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-11-24T18:06:35+01:00', 'author': 'Frederik Nygaard Pedersen', 'moddate': '2025-11-24T18:06:35+01:00', 'source': 'e:\\Tanmoy Saha\\Upskill\\Tech Stack\\Projects\\Document Portal\\notebook\\data\\sample.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='1 \n \n \n \n  \nCopenhagen Business School \nSynopsis \nDatafication: Regulation, Governance, Security, Privacy and Ethics \n  \n Is Meta’s ‘pay-or-consent’ model compatible with \nthe GDPR’s consent standard, or is it a form of \ncoercion that undermines data protection as a \nfundamental right? \n  \nExam : Datafication: Regulation, Governance, Security, Privacy and Ethics \n(CDSCO2002E) - Oral exam based on written product (IC) \nExaminers : Nicole Stybnarova \nProgram : MSc Business Administration and Data Science \nTeam Members : Frederik Pedersen (168272), Kristian Jensen (185917), Tanmoy Saha (182422)

In [ ]:
len(documents)

13

In [ ]:
## this is an experimental value. There is no specific chunk size.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len,
)

In [ ]:
docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

95

In [ ]:
docs

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-11-24T18:06:35+01:00', 'author': 'Frederik Nygaard Pedersen', 'moddate': '2025-11-24T18:06:35+01:00', 'source': 'e:\\Tanmoy Saha\\Upskill\\Tech Stack\\Projects\\Document Portal\\notebook\\data\\sample.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='1 \n \n \n \n  \nCopenhagen Business School \nSynopsis \nDatafication: Regulation, Governance, Security, Privacy and Ethics \n  \n Is Meta’s ‘pay-or-consent’ model compatible with \nthe GDPR’s consent standard, or is it a form of \ncoercion that undermines data protection as a \nfundamental right? \n  \nExam : Datafication: Regulation, Governance, Security, Privacy and Ethics \n(CDSCO2002E) - Oral exam based on written product (IC) \nExaminers : Nicole Stybnarova'),
 Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-11-24T18:06:35+01:00', 'author':

In [ ]:
docs[0]

Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-11-24T18:06:35+01:00', 'author': 'Frederik Nygaard Pedersen', 'moddate': '2025-11-24T18:06:35+01:00', 'source': 'e:\\Tanmoy Saha\\Upskill\\Tech Stack\\Projects\\Document Portal\\notebook\\data\\sample.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='1 \n \n \n \n  \nCopenhagen Business School \nSynopsis \nDatafication: Regulation, Governance, Security, Privacy and Ethics \n  \n Is Meta’s ‘pay-or-consent’ model compatible with \nthe GDPR’s consent standard, or is it a form of \ncoercion that undermines data protection as a \nfundamental right? \n  \nExam : Datafication: Regulation, Governance, Security, Privacy and Ethics \n(CDSCO2002E) - Oral exam based on written product (IC) \nExaminers : Nicole Stybnarova')

In [ ]:
docs[0].metadata

{'producer': 'Microsoft® Word 2021',
 'creator': 'Microsoft® Word 2021',
 'creationdate': '2025-11-24T18:06:35+01:00',
 'author': 'Frederik Nygaard Pedersen',
 'moddate': '2025-11-24T18:06:35+01:00',
 'source': 'e:\\Tanmoy Saha\\Upskill\\Tech Stack\\Projects\\Document Portal\\notebook\\data\\sample.pdf',
 'total_pages': 13,
 'page': 0,
 'page_label': '1'}

In [ ]:
docs[0].page_content

'1 \n \n \n \n  \nCopenhagen Business School \nSynopsis \nDatafication: Regulation, Governance, Security, Privacy and Ethics \n  \n Is Meta’s ‘pay-or-consent’ model compatible with \nthe GDPR’s consent standard, or is it a form of \ncoercion that undermines data protection as a \nfundamental right? \n  \nExam : Datafication: Regulation, Governance, Security, Privacy and Ethics \n(CDSCO2002E) - Oral exam based on written product (IC) \nExaminers : Nicole Stybnarova'

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
vector_store = FAISS.from_documents(docs, embedding_model)

In [ ]:
embedding_model.embed_documents(docs[0].page_content)

In [ ]:
len(embedding_model.embed_documents(docs[0].page_content)[0])

In [ ]:
relevant_doc = vector_store.similarity_search("What is the research question here?", k=3)

In [ ]:
relevant_doc[0].page_content

In [ ]:
relevant_doc[1].page_content

In [ ]:
relevant_doc[2].page_content

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
retriever.invoke("What is the research question here?", k=3)

In [ ]:
prompt_template = """
    Answer the question based on the context provided below.
    If the context does not contain sufficient information, respond with:
    "I do not have enough information about this."

    Context: {context}

    Question: {question}

    Answer:"""

In [ ]:
from langchain_core.prompts import PromptTemplate


In [ ]:
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

In [ ]:
prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser = StrOutputParser()

In [ ]:
rag_chain = prompt | llm | parser

In [ ]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])


In [ ]:
rag_chain = (
    {
        "context": retriever | format_docs, "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
rag_chain.invoke("what is section 3 about")